In [2]:
#ODM stands of Object Document mapper
#Just like ORM like SQLAlchemy was for relational databases. ODM is for document databases like MongoDB

# It maps documents to python objects.
# MongoEngine is the ODM we will work with in our day to day lives. similar to Django ORM API

# Data modelling with document model classes.

# We will use python now to model the database instead of JSON

#ORMs work with realtional databases. These database mostly are 2 diemnsional tables. This allows SQLAclhemy to be used with multiple relational databases.
# ODMs like MongoEngine usually works for NoSQL database, specfically MongoDb

#MongoEngine API is very similar to Django ORM API. However, it is not a replacement for Django ORM API

#MongoEngine does not do anything to the MongoDB database. You could still interoperably use any other tool with MonoDb

#Install MongoEngine

%pip install mongoengine

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#Just like SQLAlchemy where you had to create a base class from DerivativeClass to then subclass inot model classes.
# mongoengine also has a similar concept. You need to subclass Document class to start modeling collections,
# Unlike SQLAlchemy, you can directly use subclass to model collection.

from mongoengine import Document
#Field types for Mongo DB to immport from MongoEngine
from mongoengine import fields #fields submodule has most of the MongoDB Field types
import datetime

# will create a collection by name 'investments' by default in the Db
class Investment(Document):
    coin = fields.StringField(max_length=32)
    currency = fields.StringField(max_length=3)
    amount = fields.FloatField(min_value=0.00001)
    timestamp = fields.DateTimeField(default=datetime.datetime.now())
    sell = fields.BooleanField(default=False)
    
#Above was a simple model class using Mongoengine

#Let's now create a document from it
bitcoin = Investment(coin="bitcoin", currency="USD", amount=1.0)

#Adding to the database is just to call the save() method, it's that easy
bitcoin.save()

#To retreive the document from database we can use a class attribute from the document model called "objects".
# The objects class attribute is special attribute that can return all the instances of the documemt

#Retreives all documents
for investment in Investment.objects:
    print(investment.coin)
    
#query a document

usd_investment = Investment.objects.filter(currency="USD")
print(usd_investment)

#There are numerous operators that you append to the field name using double underscore
small_investment = Investment.objects.filter(amount__lt-1.0) #__lt is appended to the fieldname amount. this means return doucments that have amounts less that 1.0

#If you want to update an document. you can call update method, no need to call save()
usd_investment.update(amount=1.5)

#delete method will remove the document, again no need to call save()

usd_investment.delete()

ConnectionFailure: You have not defined a default connection

In [8]:
#Portfolio manager with MongoEngine

# We will reuse the mongita implementation of the crypto currency manager to work with MongoDB.
#We just need to switch the client to use MongoDB and that's it.



import requests
import click
from mongoengine import Document, fields, connect
import datetime

api_key = "CG-YgaSDYyvsiDQsZvTB8KNjJcR"

headers = {
    "x-cg-demo-api-key": api_key
}

def get_coin_price(coin_id, currency):
    url = f"https://api.coingecko.com/api/v3/simple/price?ids={coin_id}&vs_currencies={currency}"
    data = requests.get(url, headers=headers).json()
    return data[coin_id][currency]


class Investment(Document):
    coin = fields.StringField(max_length=32)
    currency = fields.StringField(max_length=3)
    amount = fields.FloatField(min_value=0.00001)
    timestamp = fields.DateTimeField(default=datetime.datetime.now())
    sell = fields.BooleanField(default=False)
    
    def __str__(self):
        return f"<Investment | coin: {self.coin}, currency: {self.currency}, amount: {self.amount}>"

#Helper functions
def _select_investment():
    investment_coins = Investment.objects.all().fields(coin=1) #Just like projection document in pymongo, determines how the output will be projected, field(coin=1) tells to just return a single field.
    for index,coin in enumerate(investment_coins):
        print(f"{index + 1}: {coin.coin}")
    selected_coin_index = int(input("Select an investment:")) - 1
    selected_coin_oid = investment_coins[selected_coin_index].id #Just like in pymongo we had "_id" dictionary key that represented the id of the docuemnt, in MongoEngine it will be an inbuilt attribute of the model class
    return Investment.objects(id=selected_coin_oid).first() #The first method returns the obkect.




connect("engine_portfolio") #just like pymongo the database is create on first use. default database it will look to is localhost:27017

    
@click.group()
def cli():
    pass

@cli.command(help="helper command to clear the database")
def clear_data():
    Investment.drop_collection() #Clears data from collection

@cli.command(help="Seed the database.")
@click.option("--force", is_flag=True, help="Seed even if database is not empty.")
def seed_data(force):
    data = [
        ("bitcoin", "USD", 1.0, False),
        ("ethereum", "GBP", 10.0, True),
        ("dogecoin", "EUR", 100.0, False)
    ]
    for row in data:
        obj = Investment(
            coin=row[0],
            currency=row[1],
            amount=row[2],
            sell=row[3]
        )
        obj.save()

@cli.command(help="See the detail of an investment")
def view_investment():
    selected_investment = _select_investment()
    coin_price = get_coin_price(selected_investment.coin, selected_investment.currency.lower())

    print(f"You {'bought' if not selected_investment.sell else 'sold'} {selected_investment.amount} {selected_investment.coin} for {selected_investment.amount * coin_price} {selected_investment.currency}")
    
    
@cli.command()
@click.option("--coin_id")
@click.option("--currency")
@click.option("--amount", type=float)
@click.option("--sell", is_flag=True)
def add_investments(coin_id, currency, amount, sell):
    try:
        investment = Investment(
            coin=coin_id,
            currency=currency,
            amount=amount,
            sell=sell
        )
        investment.save()
        print(f"Added {'sell' if sell else 'buy'} of {amount} {coin_id} in {currency}")
    except Exception as e:
        print(f"An error occurred: {e}")
        
@cli.command()
def show_db():
    results = investments.find({})
    [print(result) for result  in results]

# Function to run commands in Jupyter
def run_command(command, *args):
    try:
        cli.main(args=[command] + list(args), standalone_mode=False)
    except SystemExit:
        pass

# Example usage in Jupyter
# run_command('clear-data')
# run_command('seed-data')
# run_command('add-investments', '--coin_id', 'solana', '--currency', 'inr', '--amount', '1000.0')
run_command('view-investment')
# run_command('add-investments', '--coin_id', 'bitcoin', '--currency', 'inr', '--amount', '0.5', '--sell')

# #Getting investment value
# run_command('get-investment-value', '--coin_id', 'bitcoin', '--currency', 'inr')

# run_command('show-db')

1: bitcoin
2: ethereum
3: dogecoin
4: solana
You bought 1000.0 solana for 13234950.0 inr


In [ ]:
#Embedded documents and lists

#for model that is to be embedded in another doucment or be in a list, the document model class should derive from EmbeddedDocument
#If you try to embed document class you saw earlier, it will throw an error

from mongoengine import Document, EmbeddedDocument, fields
import datetime

class WatchlistMetadata(EmbeddedDocument):
    #now model embedded fields as needed like shown below
    currency= fields.StringField(max_length=3)
    description= fields.StringField()
    date_created= fields.DateField(default=datetime.datetime.now().date)
    

class WatchlistCoins(EmbeddedDocument):
    coin= fields.StringField(max_length=32)
    note= fields.StringField()
    date_added = fields.DateField(default=datetime.datetime.now().date)
    
class Watchlist(Document):
    name = fields.StringField(max_length=256)
    metadata = fields.EmbeddedDocumentField(WatchlistMetadata) #Metadata is an embedded document field, we have to give it a EmbeddedDocument model class type to expect
    coins= fields.EmbeddedDocumentListField(WatchlistCoins) #coins is an list of embedded documents of type Watchcoin
    
#creation of embedded document is no different than creating a document 
metadata = WatchlistMetadata(description="It's a watchlist", currency="USD")

#Assigning an embedded document to a doucment
watchlist = Watchlist(name="My Watchlist", metadata=metadata, coins=[])

#Embedded document list fieilds support methods like append

watchlist.coins.append(WatchlistCoins(coin="dogecoin", note="It's a joke"))

watchlist.save() #This will save all the document, EmbeddedDocuementsFields and EmbeddedDocuemntListFields

#Querying embedded documents

one_week = datetime.datetime.now().date - datetime.timedelta(days=7)

recent_lists = watchlist.objects(metadata__date_created__gte=one_week) #The filter passed to the objects was like this DocumentField__EmbeddedDocumentField__operator = value

#Querying embedded document fields
bitcoin_watchlist = watchlist.objects(coins__coin="bitcoin").first() # coins__coin is equivalent to saying coins.coin first() returns the first Document that matched the filter.



TypeError: db_field should be a string.